In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Retinoblastoma"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Retinoblastoma/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Retinoblastoma/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Retinoblastoma/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Retinoblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Search for directories related to Retinoblastoma
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)

# Retinoblastoma is an eye cancer, so look for ocular/eye cancer datasets
matching_dirs = [dir_name for dir_name in tcga_dirs 
                if any(term in dir_name.lower() for term in 
                       ["retinoblastoma", "eye", "ocular", "uveal"])]

if not matching_dirs:
    print(f"No exact matching directory found for trait: {trait}")
    print(f"Available directories: {tcga_dirs}")
    
    # Record that this trait is not available and exit
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    print(f"Task marked as completed. {trait} is not directly represented in the TCGA dataset.")
else:
    # If we found matching directories
    print(f"Found matching directories: {matching_dirs}")
    
    # Select the most appropriate directory that might contain retinoblastoma data
    selected_dir = matching_dirs[0]  # Default to first match
    if "TCGA_Ocular_melanomas_(UVM)" in matching_dirs:
        selected_dir = "TCGA_Ocular_melanomas_(UVM)"  # This is likely the closest match for eye cancer
    
    print(f"Selected directory: {selected_dir}")
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Get file paths for clinical and genetic data
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Step 3: Load the files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # Step 4: Print column names of clinical data
    print("Clinical data columns:")
    print(clinical_df.columns.tolist())


Found matching directories: ['TCGA_Ocular_melanomas_(UVM)']
Selected directory: TCGA_Ocular_melanomas_(UVM)


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'clinical_M', 'clinical_N', 'clinical_T', 'clinical_stage', 'cytogenetic_abnormality', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'extranocular_nodule_size', 'extrascleral_extension', 'extravascular_matrix_patterns', 'eye_color', 'form_completion_date', 'gender', 'gene_expression_profile', 'height', 'histological_type', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consent_verified', 'initial_pathologic_diagnosis_method', 'initial_weight', 'is_ffpe', 'lost_follow_up', 'metastatic_site', 'mitotic_count', 'new_neoplasm_event_occurrence_anatomi

### Step 2: Find Candidate Demographic Features

In [3]:
# 1. Identify candidate columns for age and gender
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# 2. Load clinical data to preview candidate columns
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Ocular_melanomas_(UVM)')
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].iloc[:5].tolist()

# Preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].iloc[:5].tolist()

print("Age columns preview:")
print(age_preview)
print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [47, 56, 54, 51, 76], 'days_to_birth': [-17514, -20539, -19894, -18948, -28025]}

Gender columns preview:
{'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Inspecting the candidate demographic columns

# Selecting the age column
# Both age_at_initial_pathologic_diagnosis and days_to_birth contain meaningful values
# age_at_initial_pathologic_diagnosis directly gives the age in years, which is more interpretable
age_col = "age_at_initial_pathologic_diagnosis"

# Selecting the gender column
# The 'gender' column contains clear gender information (MALE, FEMALE)
gender_col = "gender"

# Print the information for the chosen columns
print(f"Selected age column: {age_col}")
print(f"Age column preview: {{'age_at_initial_pathologic_diagnosis': [47, 56, 54, 51, 76]}}")

print(f"Selected gender column: {gender_col}")
print(f"Gender column preview: {{'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'MALE']}}")


Selected age column: age_at_initial_pathologic_diagnosis
Age column preview: {'age_at_initial_pathologic_diagnosis': [47, 56, 54, 51, 76]}
Selected gender column: gender
Gender column preview: {'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'MALE']}


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Use the Ocular melanomas directory identified in Step 1
selected_dir = "TCGA_Ocular_melanomas_(UVM)"
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for ocular melanoma patients from TCGA, which is the closest available dataset for studying Retinoblastoma as both are eye-related cancers."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 80 samples
After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (80, 19851) (samples x features)


After handling missing values, data shape: (80, 19851)
Quartiles for 'Retinoblastoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Retinoblastoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 51.0
  50% (Median): 61.5
  75%: 74.25
Min: 22
Max: 86
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 35 occurrences. This represents 43.75% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset was determined to be unusable and was not saved.
